In [2]:
import re
from tqdm import tqdm

import setup

In [5]:
rexp = r"Testing algorithm (?P<alg_num>\d+)\((?P<alg_name>[a-zA-Z0-9\-]+)\): \'(?P<creator>[^\']*)\' ->\'(?P<signer>[^\']*)\' -> \'(?P<verifier>[^\']*)\' \((?P<alg_size>\d+) bits\) Signature \& verify ok, create (?P<time_create_usec>\d+)usec, signature (?P<time_sign_usec>\d+)usec, verify (?P<time_verify_usec>\d+)usec"
    
raw_data = [
    re.match(rexp, l).groupdict()
    for _ in tqdm(range(100))
    for l in setup.auth("test-algorithms").split("\n") if l
]


stdout: /root/dns-falcon

stderr: 


In [6]:
import pandas as pd

stdout: 
stderr: 


In [7]:
data = pd.DataFrame(raw_data)
for c in ['alg_num', 'alg_size', 'time_create_usec', 'time_sign_usec', 'time_verify_usec']:
    data[c] = data[c].apply(int)
data

stdout: /root/dns-falcon

stderr: 


In [ ]:
mdata = data.melt(
    id_vars=['alg_num', 'alg_name', 'alg_size', 'creator', 'signer', 'verifier'], 
    value_vars=['time_create_usec', 'time_sign_usec', 'time_verify_usec'], 
    value_name='duration_usec'
)
mdata

In [ ]:
mdata['action'] = mdata['variable'].apply(lambda v: v.split('_')[1])
mdata['library'] = mdata.apply(lambda row: row[{'create': 'creator', 'sign': 'signer', 'verify': 'verifier'}[row['action']]], axis=1)
mdata['library'] = mdata['library'].replace({'OpenSSL PQC': 'OQS/OpenSSL'})
mdata['library'] = mdata['library'].apply(lambda lib: lib.split(' ', 1)[0])
mdata['library'] = mdata['library'].replace({'OpenSSL': 'OQS/OpenSSL'})
mdata['algorithm'] = mdata.apply(lambda row: f"{row['alg_name']} {row['alg_size']}bits\n[{row['library']}]", axis=1)
mdata[['library', 'algorithm', 'action', 'duration_usec']]

In [ ]:
import seaborn as sns

sns.set(font_scale=2)
sns.set_style('whitegrid')

In [ ]:
ALGORITHM_COLORS = {
    'RSASHA1 2048bits\n[OQS/OpenSSL]': '#ff0000',
    'RSASHA1-NSEC3-SHA1 2048bits\n[OQS/OpenSSL]': '#dd0000',
    'RSASHA256 2048bits\n[OQS/OpenSSL]': '#bb0000',
    'RSASHA512 2048bits\n[OQS/OpenSSL]': '#990000',
    'ECDSAP256SHA256 256bits\n[OQS/OpenSSL]': '#00ff00',
    'ECDSAP384SHA384 384bits\n[OQS/OpenSSL]': '#00dd00',
    'ED25519 256bits\n[OQS/OpenSSL]': '#0000ff',
    'ED25519 256bits\n[Sodium]': '#0000dd',
    'ED448 456bits\n[OQS/OpenSSL]': '#0000bb',
    'falcon 10248bits\n[OQS/OpenSSL]': '#00ffff',
}
ALGORITHM_ORDER = list(ALGORITHM_COLORS.keys())
ALGORITHM_PALETTE = sns.color_palette([ALGORITHM_COLORS[a] for a in ALGORITHM_ORDER])

In [ ]:
g = sns.displot(
    data=mdata,
    x='duration_usec',
    row='algorithm', col='action',
    hue='algorithm', hue_order=ALGORITHM_ORDER, palette=ALGORITHM_PALETTE,
    log_scale=(True, False),
    height=.9,
    aspect=6/.9,
    facet_kws=dict(legend_out=True, margin_titles=False, despine=False, sharey=False),
    kind='hist',
    kde=True,    
)
g.set(
    #xlim=(1*1e-3, 1e-1), 
    ylabel='', 
    yticks=[],
    xticks=[10, 100, 1000, 10000],
    xticklabels=["10µs", "100µs", "1ms", "10ms"],
    title='',
)

for ax, label in zip(g.axes[-1, :], ['Create Keypair', 'Sign', 'Verify']):
    ax.set_xlabel(f'{label}\nDuration')
    
g.figure.suptitle('Crypto Algorithm Run Time as Measured by PowerDNS', y=1.02)
g.figure.subplots_adjust(wspace=0, hspace=0)
g.legend.set_title('')
g.legend.set_zorder(1)
g.savefig('data/crypto_times_indv.pdf')